In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lit
import locale
import sys
import os
import re

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [57]:
spark

# Leyendo lista de Colegios y Estudiantes

In [87]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [88]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [89]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [90]:
print("Archivos de estudiantes: ", estuDic['20181'],estuDic['20182'])
print("Archivos de colegios: ",coleDic['20181'],coleDic['20182'])

Archivos de estudiantes:  SB11_20181.txt SB11_20182.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-20181.txt SB11-CLASIFI-PLANTELES-20182.txt


In [91]:
df1 = spark.read.load(base2 + '/' + estuDic['20181'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [92]:
df1.columns

['estu_tipodocumento',
 'estu_nacionalidad',
 'estu_genero',
 'estu_fechanacimiento',
 'periodo',
 'estu_consecutivo',
 'estu_estudiante',
 'estu_pais_reside',
 'estu_tieneetnia',
 'estu_etnia',
 'estu_limita_motriz',
 'estu_limita_invidente',
 'estu_limita_condicionespecial',
 'estu_limita_sordo',
 'estu_limita_sdown',
 'estu_limita_autismo',
 'estu_depto_reside',
 'estu_cod_reside_depto',
 'estu_mcpio_reside',
 'estu_cod_reside_mcpio',
 'fami_estratovivienda',
 'fami_personashogar',
 'fami_cuartoshogar',
 'fami_educacionpadre',
 'fami_educacionmadre',
 'fami_trabajolaborpadre',
 'fami_trabajolabormadre',
 'fami_tieneinternet',
 'fami_tieneserviciotv',
 'fami_tienecomputador',
 'fami_tienelavadora',
 'fami_tienehornomicroogas',
 'fami_tieneautomovil',
 'fami_tienemotocicleta',
 'fami_tieneconsolavideojuegos',
 'fami_numlibros',
 'fami_comelechederivados',
 'fami_comecarnepescadohuevo',
 'fami_comecerealfrutoslegumbre',
 'fami_situacioneconomica',
 'estu_dedicacionlecturadiaria',
 'est

In [93]:
columnasPuntos20181 = [
 'cole_cod_dane_establecimiento',
 'cole_nombre_establecimiento',
 'punt_lectura_critica',
 'punt_matematicas',
 'punt_c_naturales',
 'punt_sociales_ciudadanas',
 'punt_ingles']

In [94]:
df1 = df1.select(columnasPuntos20181)

In [95]:
# SE leen los dos archivos y se crean dos dataframes
df2 = spark.read.load(base2 + '/' + estuDic['20182'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [96]:
df2.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_TIENEETNIA',
 'ESTU_ETNIA',
 'ESTU_LIMITA_MOTRIZ',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_TRABAJOLABORPADRE',
 'FAMI_TRABAJOLABORMADRE',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENESERVICIOTV',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEHORNOMICROOGAS',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEMOTOCICLETA',
 'FAMI_TIENECONSOLAVIDEOJUEGOS',
 'FAMI_NUMLIBROS',
 'FAMI_COMELECHEDERIVADOS',
 'FAMI_COMECARNEPESCADOHUEVO',
 'FAMI_COMECEREALFRUTOSLEGUMBRE',
 'FAMI_SITUACIONECONOMICA',
 'ESTU_DEDICACIONLECTURADIARIA',
 'ESTU_DEDICACIONINTERNET',
 'ESTU_HORASSEMANATRABAJA',
 'ESTU_TIPOREMUNERACION',
 'COLE_CODIGO_ICFES',
 'COLE_COD_DANE_ESTABLECIMIEN

In [97]:
columnasPuntos20182 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_LECTURA_CRITICA',
 'PUNT_MATEMATICAS',
 'PUNT_C_NATURALES',
 'PUNT_SOCIALES_CIUDADANAS',
 'PUNT_INGLES',
 ]

In [98]:
df2 = df2.select(columnasPuntos20182)

In [99]:
cols = df1.columns
cols.remove('cole_cod_dane_establecimiento')
cols.remove('cole_nombre_establecimiento')
cols

['punt_lectura_critica',
 'punt_matematicas',
 'punt_c_naturales',
 'punt_sociales_ciudadanas',
 'punt_ingles']

In [100]:
for cp in cols:
    df1 = df1.withColumn(cp, regexp_replace(cp, ',', '.'))
    df1 = df1.withColumn(cp, df1[cp].cast("float"))
    df1 = df1.na.fill({cp:0})

In [101]:
df1 = df1.withColumn('cole_cod_dane_establecimiento', df1['cole_cod_dane_establecimiento'].cast("string"))

In [102]:
cols2 = df2.columns
cols2.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols2.remove('COLE_NOMBRE_ESTABLECIMIENTO')

In [103]:
for cp in cols2:
    df2 = df2.withColumn(cp, regexp_replace(cp, ',', '.'))
    df2 = df2.withColumn(cp, df2[cp].cast("float"))
    df2 = df2.na.fill({cp:0})
df2 = df2.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df2['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [104]:
print("Numero de filas primero semestre ", df1.count())
print("Numero de filas segundo semestre ", df2.count())

Numero de filas primero semestre  19798
Numero de filas segundo semestre  549934


In [105]:
print(df1.printSchema())

root
 |-- cole_cod_dane_establecimiento: string (nullable = true)
 |-- cole_nombre_establecimiento: string (nullable = true)
 |-- punt_lectura_critica: float (nullable = false)
 |-- punt_matematicas: float (nullable = false)
 |-- punt_c_naturales: float (nullable = false)
 |-- punt_sociales_ciudadanas: float (nullable = false)
 |-- punt_ingles: float (nullable = false)

None


In [106]:
print(df2.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- PUNT_LECTURA_CRITICA: float (nullable = false)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_C_NATURALES: float (nullable = false)
 |-- PUNT_SOCIALES_CIUDADANAS: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)

None


In [107]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst1 = df1.groupBy('cole_cod_dane_establecimiento').count().orderBy('count',ascending=
False)
dsCountEst2 = df2.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)

In [108]:
dsCountEst1.show(10,False)
# Numero de estudiantes por instituciòn primer semestre

+-----------------------------+-----+
|cole_cod_dane_establecimiento|count|
+-----------------------------+-----+
|376001007670                 |180  |
|305001022682                 |163  |
|376001001116                 |159  |
|376001028219                 |147  |
|376001003461                 |139  |
|311001109605                 |130  |
|311848001011                 |127  |
|376001000276                 |127  |
|376001026925                 |123  |
|376001000314                 |122  |
+-----------------------------+-----+
only showing top 10 rows



In [109]:
dsCountEst1.count()

746

In [110]:
dsCountEst2.show(10,False)
# Numero de estudiantes por instituciòn segundo semestre

+-----------------------------+-----+
|COLE_COD_DANE_ESTABLECIMIENTO|count|
+-----------------------------+-----+
|105001000108                 |865  |
|176001800052                 |751  |
|105001013340                 |721  |
|405001016946                 |671  |
|105615000236                 |658  |
|305001017361                 |627  |
|311001105863                 |619  |
|111001024732                 |604  |
|313001006701                 |545  |
|176001005813                 |535  |
+-----------------------------+-----+
only showing top 10 rows



In [111]:
#Se crea una columna con el total de los puntajes 

df1= df1.withColumn("Suma",df1['punt_lectura_critica']+df1['punt_matematicas']+df1['punt_c_naturales']+
                    df1['punt_sociales_ciudadanas']+df1['punt_ingles'])

df2= df2.withColumn("Suma",df2['PUNT_LECTURA_CRITICA']+df2['PUNT_MATEMATICAS']+df2['PUNT_C_NATURALES']+
                    df2['PUNT_SOCIALES_CIUDADANAS']+df2['PUNT_INGLES'])

## Resultados finales por instituciòn 1er semestre

In [112]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window


df1 = df1.groupBy('cole_cod_dane_establecimiento','cole_nombre_establecimiento').mean()
df1 = df1.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df1 = df1.orderBy('rank',ascending= True)

In [113]:
df1.select('cole_cod_dane_establecimiento','cole_nombre_establecimiento','avg(Suma)','rank').show(10,False)

+-----------------------------+----------------------------------+------------------+----+
|cole_cod_dane_establecimiento|cole_nombre_establecimiento       |avg(Suma)         |rank|
+-----------------------------+----------------------------------+------------------+----+
|368276000826                 |COL LA QUINTA DEL PUENTE          |403.2631578947368 |1   |
|368001000516                 |COL NUEVO CAMBRIDGE               |403.1276595744681 |2   |
|376001013441                 |COLEGIO BILINGUE DIANA OESE       |392.5208333333333 |3   |
|368001006301                 |COLEGIO HARVARD                   |391.0             |4   |
|311769000165                 |COL SAN JORGE DE INGLATERRA       |389.2987012987013 |5   |
|311001107289                 |GIMN ALESSANDRO VOLTA             |385.2857142857143 |6   |
|311848002424                 |COL SAN CARLOS                    |382.2783505154639 |7   |
|376001043617                 |COLEGIO PHILADELPHIA INTERNACIONAL|380.72727272727275|8   |

## Resultados finales por instituciòn 2do semestre

In [114]:
df2 = df2.groupBy('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO').mean()
df2 = df2.withColumn("rank", dense_rank().over(Window.orderBy(desc("INDICE_TOTAL"))))
df2 = df2.orderBy('rank',ascending= True)

In [115]:
df2.select('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','rank').show(10,False)

+-----------------------------+----------------------------------------------------------------------------------------------------+-----------------+----+
|COLE_COD_DANE_ESTABLECIMIENTO|COLE_NOMBRE_ESTABLECIMIENTO                                                                         |avg(Suma)        |rank|
+-----------------------------+----------------------------------------------------------------------------------------------------+-----------------+----+
|305001026041                 |COL HERALDOS DEL EVANGELIO                                                                          |399.0            |1   |
|311001065489                 |LIC CAMPO DAVID                                                                                     |391.0            |2   |
|354518001139                 |CAMBRIDGE SCHOOL (GIMN BILING PLAZA SESAMO)                                                         |386.8            |3   |
|311001076634                 |COL NUEVO COLOMBO AMERICANO      

## Colegios primer semestre

In [160]:
# SE crea Dataframe de colegios primer semestre
dfc = spark.read.load(base + '/' + coleDic['20181'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_COD_DANE', dfc['COLE_COD_DANE'].cast("string"))
dfc = dfc.withColumn("rank", dense_rank().over(Window.orderBy(desc("INDICE_TOTAL"))))
dfc = dfc.orderBy('rank',ascending= True)
dfc = dfc.withColumnRenamed("rank","rank_indice")

In [161]:
dfc.columns

['PERIODO',
 '|1',
 'COLE_COD_DANE',
 '|3',
 'COLE_INST_NOMBRE',
 '|5',
 'COLE_CODMPIO_COLEGIO',
 '|7',
 'COLE_MPIO_MUNICIPIO',
 '|9',
 'COLE_COD_DEPTO',
 '|11',
 'COLE_DEPTO_COLEGIO',
 '|13',
 'COLE_NATURALEZA',
 '|15',
 'COLE_GRADO',
 '|17',
 'COLE_CALENDARIO_COLEGIO',
 '|19',
 'COLE_GENEROPOBLACION',
 '|21',
 'MATRICULADOS_ULTIMOS_3',
 '|23',
 'EVALUADOS_ULTIMOS_3',
 '|25',
 'INDICE_MATEMATICAS',
 '|27',
 'INDICE_C_NATURALES',
 '|29',
 'INDICE_SOCIALES_CIUDADANAS',
 '|31',
 'INDICE_LECTURA_CRITICA',
 '|33',
 'INDICE_INGLES',
 '|35',
 'INDICE_TOTAL',
 '|37',
 'COLE_CATEGORIA',
 'rank_indice']

In [128]:
colsColegio = dfc.columns

In [162]:
dfc.select('COLE_COD_DANE','COLE_INST_NOMBRE','INDICE_TOTAL','rank_indice').orderBy('rank_indice',ascending= True).show(10)

+-------------+--------------------+------------+-----------+
|COLE_COD_DANE|    COLE_INST_NOMBRE|INDICE_TOTAL|rank_indice|
+-------------+--------------------+------------+-----------+
| 376001013441|COLEGIO BILINGUE ...|       0.918|          1|
| 368001000516| COL NUEVO CAMBRIDGE|      0.9165|          2|
| 311769000165|COL SAN JORGE DE ...|      0.9063|          3|
| 311848002424|      COL SAN CARLOS|       0.906|          4|
| 368276000826|COL LA QUINTA DEL...|      0.9057|          5|
| 376001043617|COLEGIO PHILADELP...|      0.9053|          6|
| 311848000278|        GIMN VERMONT|      0.9044|          7|
| 311001107289|GIMN ALESSANDRO V...|      0.9029|          8|
| 311769003342|     COL LOS NOGALES|      0.9025|          9|
| 305001015503|COLEGIO LEONARDO ...|      0.9009|         10|
+-------------+--------------------+------------+-----------+
only showing top 10 rows



In [163]:
print("El numero de colegios es: ", dfc.count())

El numero de colegios es:  281


## Colegios segundo semestre

In [164]:
# SE crea Dataframe de colegios primer semestre
dfc2 = spark.read.load(base + '/' + coleDic['20182'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')
dfc2 = dfc2.withColumn('COLE_COD_DANE', dfc2['COLE_COD_DANE'].cast("string"))
dfc2 = dfc2.withColumn("rank", dense_rank().over(Window.orderBy(desc("INDICE_TOTAL"))))
dfc2 = dfc2.orderBy('rank',ascending= True)
dfc2 = dfc2.withColumnRenamed("rank","rank_indice")

In [165]:
print(dfc2.printSchema())

root
 |-- PERIODO: integer (nullable = true)
 |-- COLE_COD_DANE: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_MUNICIPIO: string (nullable = true)
 |-- COLE_COD_DEPTO: integer (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_GRADO: integer (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- MATRICULADOS_ULTIMOS_3: integer (nullable = true)
 |-- EVALUADOS_ULTIMOS_3: integer (nullable = true)
 |-- INDICE_MATEMATICAS: string (nullable = true)
 |-- INDICE_C_NATURALES: string (nullable = true)
 |-- INDICE_SOCIALES_CIUDADANAS: string (nullable = true)
 |-- INDICE_LECTURA_CRITICA: string (nullable = true)
 |-- INDICE_INGLES: string (nullable = true)
 |-- INDICE_TOTAL: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- rank_

In [166]:
dfc2.select('COLE_COD_DANE','COLE_INST_NOMBRE','INDICE_TOTAL','rank_indice').orderBy('rank_indice',ascending= True).show(10)

+-------------+--------------------+------------+-----------+
|COLE_COD_DANE|    COLE_INST_NOMBRE|INDICE_TOTAL|rank_indice|
+-------------+--------------------+------------+-----------+
| 311001065489|LIC CAMPO DAVID  ...|      0,9143|          1|
| 311001076634|COL NUEVO COLOMBO...|      0,9094|          2|
| 311001089221|INST ALBERTO MERA...|       0,908|          3|
| 311001086320|GIMNASIO COLOMBO ...|      0,9056|          4|
| 354518001139|CAMBRIDGE SCHOOL ...|      0,9051|          5|
| 311001005176|LIC NAVARRA      ...|      0,9047|          6|
| 311848000308|COL ANGLO AMERICA...|      0,9042|          7|
| 308001001871|NUEVO COLEGIO DEL...|      0,9023|          8|
| 373001007662|COL  SAN BONIFACI...|      0,8996|          9|
| 368081000937|COL EL ROSARIO   ...|      0,8982|         10|
+-------------+--------------------+------------+-----------+
only showing top 10 rows



In [167]:
print("El numero de colegios segundo semestre: ", dfc2.count())

El numero de colegios segundo semestre:  9150


## Anàlisis comparativo entre estudiantes y colegios primer semestre

In [168]:
dfjoin = df1.join(dfc, df1.cole_cod_dane_establecimiento == dfc.COLE_COD_DANE)

In [169]:
dfjoin.columns

['cole_cod_dane_establecimiento',
 'cole_nombre_establecimiento',
 'avg(punt_lectura_critica)',
 'avg(punt_matematicas)',
 'avg(punt_c_naturales)',
 'avg(punt_sociales_ciudadanas)',
 'avg(punt_ingles)',
 'avg(Suma)',
 'rank',
 'PERIODO',
 '|1',
 'COLE_COD_DANE',
 '|3',
 'COLE_INST_NOMBRE',
 '|5',
 'COLE_CODMPIO_COLEGIO',
 '|7',
 'COLE_MPIO_MUNICIPIO',
 '|9',
 'COLE_COD_DEPTO',
 '|11',
 'COLE_DEPTO_COLEGIO',
 '|13',
 'COLE_NATURALEZA',
 '|15',
 'COLE_GRADO',
 '|17',
 'COLE_CALENDARIO_COLEGIO',
 '|19',
 'COLE_GENEROPOBLACION',
 '|21',
 'MATRICULADOS_ULTIMOS_3',
 '|23',
 'EVALUADOS_ULTIMOS_3',
 '|25',
 'INDICE_MATEMATICAS',
 '|27',
 'INDICE_C_NATURALES',
 '|29',
 'INDICE_SOCIALES_CIUDADANAS',
 '|31',
 'INDICE_LECTURA_CRITICA',
 '|33',
 'INDICE_INGLES',
 '|35',
 'INDICE_TOTAL',
 '|37',
 'COLE_CATEGORIA',
 'rank_indice']

In [170]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios: 281


In [171]:
# Se cruza la informaciòn por el nombre del colegio - se evidencia que el còdigo DANE no coincide
dfjoin.select('rank','cole_cod_dane_establecimiento','cole_nombre_establecimiento','INDICE_TOTAL','rank_indice').show(10)

+----+-----------------------------+---------------------------+------------+-----------+
|rank|cole_cod_dane_establecimiento|cole_nombre_establecimiento|INDICE_TOTAL|rank_indice|
+----+-----------------------------+---------------------------+------------+-----------+
|   1|                 368276000826|       COL LA QUINTA DEL...|      0.9057|          5|
|   2|                 368001000516|        COL NUEVO CAMBRIDGE|      0.9165|          2|
|   3|                 376001013441|       COLEGIO BILINGUE ...|       0.918|          1|
|   5|                 311769000165|       COL SAN JORGE DE ...|      0.9063|          3|
|   6|                 311001107289|       GIMN ALESSANDRO V...|      0.9029|          8|
|   7|                 311848002424|             COL SAN CARLOS|       0.906|          4|
|   8|                 376001043617|       COLEGIO PHILADELP...|      0.9053|          6|
|   9|                 311769003342|            COL LOS NOGALES|      0.9025|          9|
|  10|    

## Anàlisis comparativo entre estudiantes y colegios segundo semestre

In [173]:
dfjoin2 = df2.join(dfc2, df2.COLE_COD_DANE_ESTABLECIMIENTO == dfc2.COLE_COD_DANE)

In [174]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 2do semestre y colegios:", dfjoin2.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 2do semestre y colegios: 9148


In [175]:
dfjoin2.select('rank','COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','INDICE_TOTAL','rank_indice').show(10)

+----+-----------------------------+---------------------------+------------+-----------+
|rank|COLE_COD_DANE_ESTABLECIMIENTO|COLE_NOMBRE_ESTABLECIMIENTO|INDICE_TOTAL|rank_indice|
+----+-----------------------------+---------------------------+------------+-----------+
|   2|                 311001065489|       LIC CAMPO DAVID  ...|      0,9143|          1|
|   3|                 354518001139|       CAMBRIDGE SCHOOL ...|      0,9051|          5|
|   4|                 311001076634|       COL NUEVO COLOMBO...|      0,9094|          2|
|   5|                 311001089221|       INST ALBERTO MERA...|       0,908|          3|
|   6|                 311001086320|       GIMNASIO COLOMBO ...|      0,9056|          4|
|   7|                 311848000308|       COL ANGLO AMERICA...|      0,9042|          7|
|   8|                 468547002051|       ASPAEN GIMNASIO C...|      0,8937|         14|
|   9|                 311001005176|       LIC NAVARRA      ...|      0,9047|          6|
|  10|    